In [1]:
import pandas as pd

# Filter out the training set (is_correct_on(gpt) and !is_correct_on(llama))
### 일단 T에 대해서만

In [5]:
gpt_df = pd.read_csv('/home/yl3427/cylab/selfCorrectionAgent/result/1112_t14_gpt_test_0_outof_8runs.csv').sort_values(by="patient_filename")
llama_df = pd.read_csv('/home/yl3427/cylab/selfCorrectionAgent/result/1208_t14_llama3_kewltm_0_outof_10runs.csv').sort_values(by="patient_filename")

In [6]:
llama_df.columns

Index(['patient_filename', 't', 'text', 't14_kewltm_t_is_parsed',
       't14_kewltm_t_reasoning', 't14_kewltm_t_pred'],
      dtype='object')

In [10]:
n = 0
for i in range(len(gpt_df)):
    label = gpt_df.loc[i, 't']
    gpt = gpt_df.loc[i, 'gpt4o_t_stage']
    llama = llama_df.loc[i, 't14_kewltm_t_pred']
    if (gpt != llama) and (gpt == f"T{label+1}"):
        n += 1
        print(f"{label} {gpt} {llama}")
print(n)

2 T3 T2
1 T2 T1
1 T2 T4
2 T3 T2
0 T1 T2
2 T3 T2
0 T1 T2
0 T1 T2
0 T1 T2
2 T3 T2
2 T3 T4
0 T1 T2
0 T1 T2
0 T1 T2
2 T3 T2
1 T2 T4
0 T1 T2
1 T2 T4
2 T3 T2
2 T3 T2
1 T2 T1
0 T1 T2
1 T2 T4
1 T2 T1
1 T2 T4
2 T3 T4
1 T2 T3
2 T3 T2
0 T1 T2
0 T1 T2
1 T2 T4
2 T3 T2
1 T2 T4
0 T1 T2
1 T2 T1
1 T2 T1
0 T1 T2
2 T3 T2
0 T1 T2
2 T3 T2
0 T1 T2
2 T3 T4
0 T1 T2
1 T2 T4
2 T3 T4
2 T3 T4
2 T3 T2
0 T1 T2
2 T3 T4
0 T1 T2
2 T3 T2
1 T2 T4
0 T1 T2
2 T3 T2
0 T1 T2
0 T1 T2
2 T3 T4
2 T3 T4
0 T1 T2
0 T1 T2
1 T2 T1
0 T1 T2
1 T2 T1
0 T1 T4
2 T3 T2
2 T3 T2
1 T2 T1
0 T1 T2
2 T3 T2
1 T2 T1
2 T3 T2
0 T1 T4
72


In [ ]:
rule_generation_prompt_initial = """You are provided with a pathology report for a cancer patient.
Please review this report and determine the pathologic T stage of the patient's cancer.

Here is the report:
{report}

What is the T stage from this report? Ignore any substaging information. Please select from the following four options: T1, T2, T3, T4.

Please induce a list of rules as knowledge that help you predict the T stage for the next report. Ensure there is at least one rule for each T stage (T1, T2, T3, T4) in the list of rules. Ensure each rule is general and applicable to the specific cancer type and the AJCC's TNM Staging System, avoiding any report-specific information.
"""


In [ ]:
initial_predict_prompt_t14 = """You are provided with a pathology report for a cancer patient.
Please review this report and determine the pathologic T stage of the patient's cancer.

Here is the report:
{report}

What is your reasoning to support your T stage prediction?

What is the T stage from this report? Ignore any substaging information. Please select from the following four options: T1, T2, T3, T4.

Please induce a list of rules as knowledge that help you predict the T stage for the next report. Ensure there is at least one rule for each T stage (T1, T2, T3, T4) in the list of rules. Ensure each rule is general and applicable to the specific cancer type and the AJCC's TNM Staging System, avoiding any report-specific information.
"""
subsequent_predict_prompt_t14 = """You are provided with a pathology report for a cancer patient.
Here is a list of rules you have learned to correctly predict the T stage information:
{rules}

Please review this report and determine the pathologic T stage of the patient's cancer.

Here is the report:
{report}

What is your reasoning to support your T stage prediction?

What is the T stage from this report? Ignore any substaging information. Please select from the following four options: T1, T2, T3, T4.

What is your updated list of rules that help you predict the T stage for the next report? You can either modify the original rules or add new ones. Ensure there is at least one rule for each T stage (T1, T2, T3, T4) in the list of rules. Ensure each rule is general and applicable to the specific cancer type and the AJCC's TNM Staging System, avoiding any report-specific information.
"""